# Notebook que servira para probar el reecomendador de forma independiente

In [12]:
#Paqueterias
import os
import sys
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
import random
import time
from urllib import parse, request
import re

SyntaxError: invalid syntax (779918766.py, line 11)

In [5]:
print(os.listdir('.'))

['Recomendador.ipynb']


In [15]:
juegos = ["Bully","Carmageddon","Dead Rising III","Fallout IV: New Vegas",
"Hearts of Iron","Max Payne","Hearts of Iron","Dead Island","Battlefield III","Call of Duty IV: Modern Warfare",
"Mass Effect","Mortal Kombat III","Postal"]

def test():
	juegosTest = []
	count = 0
	while count < 5:
		n = random.randint(1,len(juegos)-1)
		test1 = juegos[n]
		if not (test1 in juegosTest) :
			juegosTest.append(test1)
			count = count + 1
	return juegosTest

In [6]:
#Variables a usar
#cargamos los datos de los juegos
movies = pd.read_csv('../data/games.csv')
#Cargamos los datos de los usuarios 
ratings = pd.read_csv('../data/ratingsG.csv')

In [8]:
movies

,movieId,title
0,1,50 Cent: Bulletproof (2010)
1,2,Assassin's Creed (2010)
2,3,Battlefield III (2011)
3,4,Battlefield IV (2014)
4,5,BlazBlue II:Continuum Shift (1998)
...,...,...
62,63,Mortal Kombat II (1995)
63,64,Mortal Kombat III (1996)
64,65,NARC (2005)
65,66,Phantasmagoria (2009)


In [9]:
ratings

,userId,movieId,rating
0,1,28,4.7
1,2,67,1.6
2,3,47,1.0
3,4,45,2.6
4,5,51,4.2
...,...,...,...
2995,942,31,0.3
2996,270,39,0.3
2997,131,9,0.7
2998,721,1,0.7


In [10]:
#Se ponen parentesis para evitar conflictos con peliculas que pueden tener años en sus titulos
movies['year'] = movies.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Quitamos los parentesis
movies['year'] = movies.year.str.extract('(\d\d\d\d)',expand=False)
#Quitamos el año de la columna title
movies['title'] = movies.title.str.replace('(\(\d\d\d\d\))', '')
#quitamos todos los espacios en blanco usando strip
movies['title'] = movies['title'].apply(lambda x: x.strip())

/var/folders/pl/m9g73fks67x0r_5pptyg_4f00000gn/T/ipykernel_94038/3969403497.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  movies['title'] = movies.title.str.replace('(\(\d\d\d\d\))', '')


In [11]:
movies.head(30)

,movieId,title,year
0,1,50 Cent: Bulletproof,2010
1,2,Assassin's Creed,2010
2,3,Battlefield III,2011
3,4,Battlefield IV,2014
4,5,BlazBlue II:Continuum Shift,1998
5,6,Blitz I:The League,1995
6,7,Blood,2000
7,8,BMX XXX,2002
8,9,Bully,2011
9,10,Call of Duty IV: Modern Warfare,2010


### Calificamos los juegos de acuerdo al usuario

In [16]:
gamesSurvey= test()

In [17]:
gamesSurvey

['Battlefield III',
 'Mortal Kombat III',
 'Postal',
 'Max Payne',
 'Hearts of Iron']

## Por cada juego se debe asignar una calificación, para poder aplicar el algoritmo.

In [21]:
gamesSurvey[0]
juego1 = input(gamesSurvey[0])
#juego1 = 5#Se puede poner manualmente si es mas comodo

In [22]:
gamesSurvey[1]
juego2 = input(gamesSurvey[1])

In [23]:
gamesSurvey[2]
juego3 = input(gamesSurvey[2])

In [24]:
gamesSurvey[3]
juego4 = input(gamesSurvey[3])

In [25]:
gamesSurvey[4]
juego5 = input(gamesSurvey[4])

In [26]:
userInput = [
            {'title':gamesSurvey[0], 'rating':float(juego1)},
            {'title':gamesSurvey[1], 'rating':float(juego2)},
            {'title':gamesSurvey[2], 'rating':float(juego3)},
            {'title':gamesSurvey[3], 'rating':float(juego4)},
            {'title':gamesSurvey[4], 'rating':float(juego5)}
         ] 

In [27]:
userInput

[{'title': 'Battlefield III', 'rating': 5.0},
 {'title': 'Mortal Kombat III', 'rating': 5.0},
 {'title': 'Postal', 'rating': 5.0},
 {'title': 'Max Payne', 'rating': 5.0},
 {'title': 'Hearts of Iron', 'rating': 5.0}]

### Hacemos un filtrado

In [30]:
inputMovies = pd.DataFrame(userInput)
#Filtramos las peliculas por titulo
inputId = movies[movies['title'].isin(inputMovies['title'].tolist())]
#ordenamos las peliculas por id
inputMovies = pd.merge(inputId, inputMovies)
#Quitamos el año
inputMovies = inputMovies.drop('year',axis=1)

In [31]:
inputMovies

,movieId,title,rating
0,3,Battlefield III,5.0
1,44,Hearts of Iron,5.0
2,60,Max Payne,5.0
3,64,Mortal Kombat III,5.0
4,67,Postal,5.0


In [32]:
#Buscamos los usuarios que tambien "vieron" la pelicula
userSubset = ratings[ratings['movieId'].isin(inputMovies['movieId'].tolist())]
#quitar comentario para ver tabla
#userSubset.head()
userSubsetGroup = userSubset.groupby(['userId'])
#Ordenmos por usuarios con la pelicula mas vista 
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)
userSubsetGroup[0:3]
userSubsetGroup = userSubsetGroup[0:100]
#Usaremos la coorrelacion de pearson, la guardamos en un diccionario, la llave es el id 
#del usuario.
pearsonCorrelationDict = {}
#Para cada grupo de usuarios 
for name, group in userSubsetGroup:
    #Ordenamos la entrada y el grupo de usuarios para que no se mezclen
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    #Obtenemos N para la formula
    nRatings = len(group)
    #
    #Obtenemos Las reseñas de los usuarios con peliculas en comun
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    tempRatingList = temp_df['rating'].tolist()
    #Ponemos las reseñas del grupo en una lista
    tempGroupList = group['rating'].tolist()
    #Calculamos la correlacion de pearson entre los usuarios (x & y)
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
#Si el denomindor es 0 entonces la correlación es 0 
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

In [35]:
pearsonCorrelationDict

{438: 0,
 709: 0,
 250: 0,
 322: 0,
 393: 0.9999999999999999,
 403: 1.0,
 408: 0,
 524: 0,
 535: 0,
 555: 0,
 582: -0.9999999999999885,
 708: 0,
 796: 0,
 958: 1.0000000000000002,
 2: 0,
 6: 0,
 11: 0,
 12: 0,
 24: 0,
 26: 0,
 29: 0,
 35: 0,
 39: 0,
 40: 0,
 42: 0,
 44: 0,
 49: 0,
 60: 0,
 61: 0,
 81: 0,
 85: 0,
 90: 0,
 93: 0,
 94: 0,
 96: 0,
 97: 0,
 100: 0,
 106: 0,
 112: 0,
 113: 0,
 115: 0,
 135: 0,
 141: 0,
 143: 0,
 150: 0,
 154: 0,
 160: 0,
 162: 0,
 173: 0,
 175: 0,
 177: 0,
 178: 0,
 180: 0,
 188: 0,
 202: 0,
 203: 0,
 226: 0,
 227: 0,
 229: 0,
 238: 0,
 244: 0,
 247: 0,
 251: 0,
 254: 0,
 259: 0,
 265: 0,
 266: 0,
 271: 0,
 274: 0,
 279: 0,
 282: 0,
 283: 0,
 288: 0,
 289: 0,
 290: 0,
 291: 0,
 300: 0,
 303: 0,
 328: 0,
 329: 0,
 332: 0,
 342: 0,
 343: 0,
 347: 0,
 348: 0,
 355: 0,
 359: 0,
 365: 0,
 366: 0,
 368: 0,
 378: 0,
 383: 0,
 392: 0,
 396: 0,
 410: 0,
 414: 0,
 423: 0,
 431: 0,
 440: 0,
 443: 0}

In [33]:
#obtener las películas vistas por los usuarios en nuestro pearsonDF desde el dataframe 
#de calificaciones y luego almacenar su correlación en una nueva columna _similarityIndex ".
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
print("DF: ",pearsonDF)
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
#Obtenemos los 50 usuarios que se parecen mas 
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsersRating=topUsers.merge(ratings, left_on='userId', right_on='userId', how='inner')
#Multiplicamos la similitud por las calificaciones del usuario
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
# Aplicamos una suma de los usuarios principales después de agruparlos por ID 
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']

DF:         0
438  0.0
709  0.0
250  0.0
322  0.0
393  1.0
..   ...
414  0.0
423  0.0
431  0.0
440  0.0
443  0.0

[100 rows x 1 columns]


In [36]:
#Creamos un dataframe vacio
recommendation_df = pd.DataFrame()
#Tomamos media ponderada
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
#Obtenemos las mejores peliculas recomendadas y las ordenamos por id
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)

### Se busca recomendaciones para el usuario segun sus calificaciones

In [39]:
movies.loc[movies['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())].drop('movieId', 1)

/var/folders/pl/m9g73fks67x0r_5pptyg_4f00000gn/T/ipykernel_94038/1483887295.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  movies.loc[movies['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())].drop('movieId', 1)


,title,year
0,50 Cent: Bulletproof,2010
2,Battlefield III,2011
12,Carmageddon II: Carpocalypse Now,1998
14,Command & Conquer Generals,1999
15,Commandos I: Behind Enemy Lines,2000
24,Dead Rising,2013
55,Manhunt II,2010
59,Max Payne,2016
61,Mortal Kombat,1993
63,Mortal Kombat III,1996
